# Case Study: Conservative Retiree

[![Open in Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/engineerinvestor/monteplan/blob/main/notebooks/05_case_study_retiree.ipynb)

**Persona:** Pat, age 60. Has $700K across two accounts, plans to retire at 65. Will collect Social Security ($2,500/month) starting at 67. Prefers a conservative allocation.

**Questions:**
1. How much does Social Security improve the plan?
2. How should the allocation shift over time?
3. Does tax model choice matter?
4. What's the biggest risk?

In [ ]:
# Uncomment and run in Google Colab:
# !pip install -q "monteplan @ git+https://github.com/engineerinvestor/monteplan.git"

In [ ]:
import matplotlib.pyplot as plt
import numpy as np

from monteplan.config.defaults import conservative_retiree_plan, default_market, default_policies
from monteplan.config.schema import (
    SimulationConfig, PolicyBundle, MarketAssumptions, AssetClass,
    GlidePath, PlanConfig, AccountConfig, GuaranteedIncomeStream,
)
from monteplan.core.engine import simulate

## Pat's Plan

In [ ]:
plan = conservative_retiree_plan()
market = default_market()
policies = default_policies()
sim_config = SimulationConfig(n_paths=1000, seed=42)

print(f"Age: {plan.current_age} -> Retire: {plan.retirement_age} -> Horizon: {plan.end_age}")
print(f"Monthly spending: ${plan.monthly_spending:,.0f}")
print(f"Current portfolio: ${sum(a.balance for a in plan.accounts):,.0f}")
print(f"\nGuaranteed income:")
for gi in plan.guaranteed_income:
    print(f"  {gi.name}: ${gi.monthly_amount:,.0f}/mo starting age {gi.start_age}, COLA={gi.cola_rate:.0%}")

## Base Case: With Social Security

In [ ]:
result_with_ss = simulate(plan, market, policies, sim_config)
print(f"Success with Social Security: {result_with_ss.success_probability:.1%}")

## Impact of Social Security

What if Pat didn't have Social Security?

In [ ]:
plan_no_ss = plan.model_copy(update={"guaranteed_income": []})
result_no_ss = simulate(plan_no_ss, market, policies, sim_config)

print(f"With Social Security:    {result_with_ss.success_probability:.1%}")
print(f"Without Social Security: {result_no_ss.success_probability:.1%}")
print(f"SS impact: {result_with_ss.success_probability - result_no_ss.success_probability:+.1%}")

In [ ]:
# Compare fan charts
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(14, 5), sharey=True)

for ax, res, title in [
    (ax1, result_with_ss, f"With SS ({result_with_ss.success_probability:.1%})"),
    (ax2, result_no_ss, f"Without SS ({result_no_ss.success_probability:.1%})"),
]:
    ts = res.wealth_time_series
    ages = np.linspace(plan.current_age, plan.end_age, len(ts["p50"]))
    ax.fill_between(ages, ts["p5"], ts["p95"], alpha=0.15, color="steelblue")
    ax.fill_between(ages, ts["p25"], ts["p75"], alpha=0.3, color="steelblue")
    ax.plot(ages, ts["p50"], color="steelblue", linewidth=2)
    ax.axvline(plan.retirement_age, color="gray", linestyle="--", alpha=0.5)
    ax.set_xlabel("Age")
    ax.set_title(title)
    ax.yaxis.set_major_formatter(plt.FuncFormatter(lambda x, _: f"${x/1e6:.1f}M" if x >= 1e6 else f"${x/1e3:.0f}K"))

ax1.set_ylabel("Portfolio Value ($)")
plt.suptitle("The Impact of Social Security", fontsize=14)
plt.tight_layout()
plt.show()

## Glide Path Exploration

Should Pat shift from 70/30 stocks/bonds to a more conservative allocation over time?

In [ ]:
allocations = {
    "Static 70/30": None,
    "Glide 70/30 -> 40/60": GlidePath(start_age=60, start_weights=[0.7, 0.3], end_age=80, end_weights=[0.4, 0.6]),
    "Glide 70/30 -> 30/70": GlidePath(start_age=60, start_weights=[0.7, 0.3], end_age=80, end_weights=[0.3, 0.7]),
}

glide_results = {}
for name, gp in allocations.items():
    m = market.model_copy(update={"glide_path": gp})
    r = simulate(plan, m, policies, sim_config)
    glide_results[name] = r
    print(f"{name:30s} -> {r.success_probability:.1%}")

In [ ]:
fig, ax = plt.subplots(figsize=(10, 5))
colors = ["#1f77b4", "#ff7f0e", "#2ca02c"]
for (name, res), color in zip(glide_results.items(), colors):
    ts = res.wealth_time_series
    ages = np.linspace(plan.current_age, plan.end_age, len(ts["p50"]))
    ax.plot(ages, ts["p50"], label=f"{name} ({res.success_probability:.0%})", color=color, linewidth=2)

ax.axvline(plan.retirement_age, color="gray", linestyle="--", alpha=0.5)
ax.set_xlabel("Age")
ax.set_ylabel("Median Portfolio Value ($)")
ax.set_title("Allocation Strategies Compared")
ax.legend()
ax.yaxis.set_major_formatter(plt.FuncFormatter(lambda x, _: f"${x/1e6:.1f}M" if x >= 1e6 else f"${x/1e3:.0f}K"))
plt.tight_layout()
plt.show()

## Tax Model Comparison

Most of Pat's savings are in a traditional 401k. How much do taxes matter?

In [ ]:
policies_flat = PolicyBundle(tax_model="flat", tax_rate=0.22)
policies_federal = PolicyBundle(tax_model="us_federal", filing_status="single")

result_flat = simulate(plan, market, policies_flat, sim_config)
result_federal = simulate(plan, market, policies_federal, sim_config)

print(f"Flat 22% tax:   {result_flat.success_probability:.1%}")
print(f"US Federal tax: {result_federal.success_probability:.1%}")

## Inflation Sensitivity

Inflation is the biggest long-term risk for retirees. What if average inflation is higher?

In [ ]:
inflation_levels = [0.02, 0.025, 0.03, 0.035, 0.04, 0.045]
inflation_results = []

for inf in inflation_levels:
    m = market.model_copy(update={"inflation_mean": inf})
    r = simulate(plan, m, policies, sim_config)
    inflation_results.append(r.success_probability)
    print(f"  {inf:.1%} inflation -> {r.success_probability:.1%}")

fig, ax = plt.subplots(figsize=(8, 4))
ax.plot([x*100 for x in inflation_levels], inflation_results, "o-", color="steelblue", linewidth=2)
ax.set_xlabel("Average Annual Inflation (%)")
ax.set_ylabel("Success Probability")
ax.set_title("Inflation Sensitivity")
ax.yaxis.set_major_formatter(plt.FuncFormatter(lambda x, _: f"{x:.0%}"))
ax.axhline(0.8, color="green", linestyle="--", alpha=0.5, label="80% target")
ax.legend()
plt.tight_layout()
plt.show()

## Takeaways for Pat

1. **Social Security is a game-changer** -- it dramatically improves success rates by reducing the amount that must come from the portfolio
2. **A glide path to more bonds** slightly reduces upside but may provide more peace of mind; the impact depends on the specific allocation shift
3. **Tax model choice matters** for traditional-heavy portfolios -- the progressive bracket structure may be better or worse than a flat assumption depending on withdrawal levels
4. **Inflation is the key risk** -- a sustained 1% increase in average inflation can significantly erode the plan's viability
5. **The COLA on Social Security** partially hedges inflation risk (the 2% COLA in Pat's plan offsets ~2/3 of the 3% inflation assumption)